In [1]:
import pandas as pd
import sqlite3

<h2>Создайте подключение к базе данных с помощью библиотеки sqlite3</h2>

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

<h2>Получить схему таблицы checker</h2>

In [3]:
pd.io.sql.read_sql('pragma table_info(checker)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


<h2>Для проверки таблицы используйте только первые 10 строк</h2>

In [4]:
pd.io.sql.read_sql('select * from pageviews limit 10', conn)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


<h2>Подсчитайте, сколько строк удовлетворяют следующим условиям, используя только один запрос с любым количеством подзапросов:</h2>

1.посчитайте количество строк в таблице просмотров страниц, но только с пользователями из таблицы проверки:

    - статус = «готов», мы не хотим анализировать журналы, которые находятся в режиме проверки статуса
    - numTrials = 1, мы хотим проанализировать только первые коммиты, потому что только они могут сказать нам, когда студент начал работать над лабораторной работой
    - Названия лабораторий должны быть из списка: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’. Только они были активны во время эксперимента
2. храните в фрейме данных контрольные элементы со столбцом cnt


In [5]:
query = """
SELECT COUNT(uid) as cnt
FROM pageviews
WHERE pageviews.uid IN
    (SELECT checker.uid
    FROM checker
    WHERE status = 'ready'
    AND numTrials = 1
    AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    )
    """
pd.io.sql.read_sql(query,conn)

,cnt
0,985


<h2>закройте соединение</h2>

In [6]:
conn.close()